In [ ]:
from utils.dataset import Argo3DTemperatureDataset, FrameType

data_set = Argo3DTemperatureDataset().getFrame(FrameType.surface, time=(50, 51), lat=(160, 170), lon=(100, 110), depth=(0, 1)).reshape(10, 10)

print(data_set)

In [4]:
import numpy as np

from torch import tensor

from models.LSTM import LSTMNetwork

model = LSTMNetwork(data_set.shape).to('cuda')

print(model) 

LSTMNetwork(
  (conv1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (lstm): LSTM(10, 10)
  (pool1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (pool2): AvgPool2d(kernel_size=3, stride=3, padding=0)
  (activation1): Sigmoid()
  (activation2): ReLU()
  (fc1): Linear(in_features=10, out_features=10, bias=True)
)


In [7]:
data = tensor([[data_set]]).to('cuda')

output = model(data)

print(output)
print(output.shape)

卷积：  torch.Size([1, 1, 8, 8])
池化：  torch.Size([1, 1, 2, 2])
tensor([[[[-25.1940, -23.1222],
          [-25.3725, -23.1313]]]], device='cuda:0',
       grad_fn=<MaxPool2DWithIndicesBackward0>)
torch.Size([1, 1, 2, 2])
